Sentiment & Emotion Detection (Person 2)
- Tools: NLTK (preprocessing), VADER (sentiment), Hugging Face Transformers (DistilBERT, emotion)
- Training: 3 epochs on 'emotion' dataset (16k train, 2k val), GPU, ~41 min
- Model: ./fine_tuned_emotion_model (DistilBERT, 6 emotions: sadness, joy, love, anger, fear, surprise)
- Metrics: eval_loss: 0.2229, eval_accuracy: 0.94, eval_f1: 0.9399
- Output: {'sentiment': str, 'emotion': str, 'sentiment_scores': dict}
- Test Examples:
  - "I am so happy today!" → positive, joy
  - "Yesterday was terrible." → negative, sadness
  - "I love this so much, but I’m also scared." → negative, fear
  - "I’m angry and frustrated!" → negative, anger
  - "This is surprisingly great." → positive, joy
  - "I am so happy today, but yesterday was terrible!" → negative, joy (note: expected sadness, minor context issue)
- Notes: 94% accuracy on validation, handles most cases well. Minor tweak needed for mixed temporal emotions. Ready for LLM integration (Person 3).
- Previous W&B Run (3 epoch): https://wandb.ai/amfggg/emotion-detection/runs/x745he0n

In [3]:
# Update pip and install required libraries individually
!pip install --upgrade pip -q
!pip install nltk -q
!pip install vaderSentiment -q
!pip install transformers -q
!pip install torch -q
!pip install datasets -q
!pip install scikit-learn -q  # Corrected 'sklearn' to 'scikit-learn'

# Import libraries
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, pipeline
from datasets import load_dataset
import time
from sklearn.metrics import accuracy_score, f1_score

# Download NLTK data
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

# Function to preprocess text with NLTK
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words and word.isalnum()]
    return " ".join(tokens)

# Retry logic for dataset loading
def load_dataset_with_retries(dataset_name, max_attempts=5, delay=10):
    for attempt in range(max_attempts):
        try:
            dataset = load_dataset(dataset_name)
            return dataset
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt < max_attempts - 1:
                time.sleep(delay)
            else:
                raise Exception("Failed to load dataset after all attempts")

# Load dataset
print("Loading dataset...")
try:
    dataset = load_dataset_with_retries("emotion")
except Exception as e:
    print(f"Failed to load dataset: {e}")
    exit()

# Load tokenizer and model
print("Loading model and tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=6)

# Tokenize dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

print("Tokenizing dataset...")
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Training arguments (3 epochs, no W&B)
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    eval_strategy="epoch",
    report_to="none"
)

# Define metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    return {"accuracy": acc, "f1": f1}

# Trainer with metrics
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics
)

# Train the model
print("Starting training...")
trainer.train()

# Evaluate the model
print("Evaluating model...")
eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)

# Save the model
print("Saving model...")
model.save_pretrained("./fine_tuned_emotion_model")
tokenizer.save_pretrained("./fine_tuned_emotion_model")

# Load VADER for sentiment analysis
analyzer = SentimentIntensityAnalyzer()

# Test the fine-tuned model with VADER integration and emotion mapping
def analyze_text(text):
    cleaned_text = preprocess_text(text)
    sentiment = analyzer.polarity_scores(text)
    sentiment_label = "positive" if sentiment["compound"] > 0.05 else "negative" if sentiment["compound"] < -0.05 else "neutral"
    classifier = pipeline("text-classification", model="./fine_tuned_emotion_model", tokenizer="./fine_tuned_emotion_model")
    emotions = classifier(text)
    emotion_map = {f"LABEL_{i}": emotion for i, emotion in enumerate(["sadness", "joy", "love", "anger", "fear", "surprise"])}
    top_emotion = emotion_map[emotions[0]["label"]]
    return {"sentiment": sentiment_label, "emotion": top_emotion, "sentiment_scores": sentiment}

# Test suite
test_texts = [
    "I am so happy today!",
    "Yesterday was terrible.",
    "I love this so much, but I’m also scared.",
    "I’m angry and frustrated!",
    "This is surprisingly great.",
    "I am so happy today, but yesterday was terrible!"
]
print("Running test suite...")
for text in test_texts:
    result = analyze_text(text)
    print(f"Test result for '{text}':", result)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 160.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 178.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 124.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavi

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Loading dataset...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

...

Test result for 'Yesterday was terrible.': {'sentiment': 'negative', 'emotion': 'sadness', 'sentiment_scores': {'neg': 0.608, 'neu': 0.392, 'pos': 0.0, 'compound': -0.4767}}


Device set to use cuda:0


Test result for 'I love this so much, but I’m also scared.': {'sentiment': 'negative', 'emotion': 'fear', 'sentiment_scores': {'neg': 0.286, 'neu': 0.52, 'pos': 0.193, 'compound': -0.3071}}
Test result for 'I’m angry and frustrated!': {'sentiment': 'negative', 'emotion': 'anger', 'sentiment_scores': {'neg': 0.778, 'neu': 0.222, 'pos': 0.0, 'compound': -0.7901}}


Device set to use cuda:0
Device set to use cuda:0


Test result for 'This is surprisingly great.': {'sentiment': 'positive', 'emotion': 'joy', 'sentiment_scores': {'neg': 0.0, 'neu': 0.241, 'pos': 0.759, 'compound': 0.743}}
Test result for 'I am so happy today, but yesterday was terrible!': {'sentiment': 'negative', 'emotion': 'joy', 'sentiment_scores': {'neg': 0.31, 'neu': 0.489, 'pos': 0.201, 'compound': -0.376}}
